# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [4]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
from collections import Counter

In [5]:
dvach = open('2ch_corpus.txt').read()
news = open('lenta.txt').read()

In [6]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [7]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [8]:
# отрезаем по 50 предложений из каждого корпуса для расчёта перплексии

sentences_dvach = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_dvach_test = sentences_dvach[:50]
sentences_dvach = sentences_dvach[50:]

sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]
sentences_news_test = sentences_news[:50]
sentences_news = sentences_news[50:]

In [9]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, n=3))

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

In [10]:
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix
matrix_dvach = lil_matrix((len(bigrams_dvach), len(unigrams_dvach)))

In [11]:
id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigram_dvach)}

In [12]:
for ngram in trigrams_dvach:
    word0, word1, word3 = ngram.split()
    word2 = word0+' '+word1
    matrix_dvach[bigram2id_dvach[word2],word2id_dvach[word3]] =  (trigrams_dvach[ngram]/bigrams_dvach[word2])

In [13]:
matrix_dvach = csr_matrix(matrix_dvach)

In [14]:
matrix_news = lil_matrix((len(bigrams_news), 
                   len(unigrams_news)))
id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)} # словарь
id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}
for ngram in trigrams_news:
    word0, word1, word3 = ngram.split()
    word2 = word0+' '+word1
    matrix_news[bigram2id_news[word2],word2id_news[word3]] =  (trigrams_news[ngram]/bigrams_news[word2])

In [15]:
matrix_news = csr_matrix(matrix_news)

In [16]:
def generate(matrix, id2bigram, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
#         chosen = np.random.choice(list(range(matrix.shape[1]))) # НЕПОН
        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0]) # НЕПОН
        text.append(id2word[chosen])

        if id2word[chosen] == '<end>':
            current_idx = bigram2id[start]
        else:     
            current_idx = bigram2id [id2bigram[current_idx].split( )[1] + ' ' + id2word[chosen]]
    return ' '.join(text)

In [15]:
print(generate(matrix_dvach, id2bigram_dvach, id2word_dvach, bigram2id_dvach).replace(' <end>', '.'))

на старте если бы тот кому пригодится. и подчёркивали его статус изгоя во всём этом. когда. могу сюда резюме шлюхи с академки позвонить но сука этот рельеф у меня были жесткие загоны в плане комфорта теплоизоляции и размера. джек дениэлсовая. end game сраные рп сервера разумеется петушино hmg вог красный медведь и куча других режимов и модов. вот такие пропорции. вот не за 500 р в будни. i want to be back in the middle of the stars awaited her she dove in for a time for example a metalworking lab that


In [16]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

во время первого официального допроса я стал бы торопить события заявил сегодня в акциях протеста заявив что дети очень легко объяснить 
 глава администрации nasa дан голдин выступая на открытии проекта в впк 
 по словам кузьминова предполагают учреждение новой структуры объявили о начале работы над проектом законодательной регламентации распространения информации о том что он может произойти что-то такое что приведет к снятию экономических санкций 
 по словам председателя центризбиркома александра вешнякова 
 до этого вторника на среду заседании коллегии российского авиакосмического агентства и пообещал лично разобраться с положением о порядке принятия в российскую столицу сегодня в пресс-службе росавиакосмоса дано


In [17]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

главной причиной нынешней войны на территории турции 
 степашин отказался от участия в игре поставил за 20 минут до конца текущего года сообщил департамент по вопросам которое начнется в полдень сообщает эхо москвы во владикавказ и помещен в следственный изолятор было предпринято с разведывательной целью 
 две его результативные передачи позволили медведям обыграть тампа-бей 4 0 
 судебная неприкосновенность пиночета являлась следствием его статуса пожизненного сенатора который диктатор учредил перед своей отставкой 
 некоторые жители грозного покидали город и провести реконструкцию ряда пассажирских станций и долгое время было возбуждено уголовное дело 
 вместе с тем пояснил кушаков что средства на


In [18]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

российский министр отверг критику министра обороны россии игорь сергеев сказал что они возвращают отношения между государствами 
 если я попаду в сборную на евро-2000 я сыграю там а потом вице-президента сербской респубилки 
 при этом специальной регистрации в zshops неминуемо приводит к сбоям привели 400 тысяч пользователей других провайдеров сообщает bbc 
 министр не уточнил где именно будет заключаться полная изоляция от сети интернет 
 в этом случае позаботимся чтобы европейский союз подтвердил готовность российского руководства и работников компании 
 по данным агентства reuters около 8000 офицеров полиции патрулируют эту часть грозного легче полностью расчистить и построить заново спс предложило


In [19]:
print(generate(matrix_news, id2bigram_news, id2word_news, bigram2id_news).replace('<end>', '\n'))

в соответствии с которым контроль над корневым сервером 
 с разговора об этом заявил во вторник на пресс-конференции что у кремля есть деньги указав при этом путин специально подчеркнул что ему было возвращено личное оружие 
 травмирован и один ливанский за что арестован поэтому давать показания в качестве элементов питания использует 12 картофелин 
 по уточненным данным полученным harris interactive около трети той компенсации которая должна быть продолжена так как речь идет о чисто физической неспособности консульского отдела посольства россии и поручили вешнякову вручить ему это сделать то февраль мы проживем без заимствований у некоторых коммерческих телерадиовещателей 
 атака была


Считаем перплексию:

In [17]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N)

In [18]:
def mean_perpl(test_set):
    perpl_mean = {'dvach':[], 'news':[]}
    perpl_sum_dvach = []
    perpl_sum_news = []
    for sent in test_set:
        prob = {'dvach':[], 'news':[]}
        for ngram in ngrammer(sent,n=3):
            word0, word1, word3 = ngram.split()
            word2 = word0+' '+word1
            if word2 in bigrams_dvach and ngram in trigrams_dvach:
                prob['dvach'].append(np.log(trigrams_dvach[ngram]/bigrams_dvach[word2]))
            else:
                prob['dvach'].append(np.log(0.01))
            if word2 in bigrams_news and ngram in trigrams_news:
                prob['news'].append(np.log(trigrams_news[ngram]/bigrams_news[word2]))
            else:
                prob['news'].append(np.log(0.01))
        if perplexity(prob['dvach'])!= np.inf: 
            perpl_sum_dvach.append(perplexity(prob['dvach']))
        if perplexity(prob['news'])!= np.inf: 
            perpl_sum_news.append(perplexity(prob['news']))
    perpl_mean['dvach'] = sum(perpl_sum_dvach)/len(perpl_sum_dvach)
    perpl_mean['news'] = sum(perpl_sum_news)/len(perpl_sum_news)
    return perpl_mean

In [19]:
mean_perpl(sentences_dvach_test)

{'dvach': 114.27912201188887, 'news': 108.57862227953072}

In [20]:
mean_perpl(sentences_news_test)

{'dvach': 104.88869527621735, 'news': 101.12143530695465}

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

2. Что такое сглаживание (smoothing)?